In [ ]:
!qsmxt bids out \
    --premade 'body' \
    --do_qsm \
    --do_t2starmap \
    --do_r2starmap \
    --do_swi \
    --mask_erosions 3 0 \
    --threshold_value 25.0 \
    --slurm a_barth general \
    --auto_yes

In [1]:
import os
import glob
import nibabel as nib
import numpy as np
import enum
import json
import SimpleITK as sitk
from scipy.ndimage import label
from scipy.ndimage import binary_dilation, binary_erosion, generate_binary_structure, label, center_of_mass
import pandas as pd

/home/uqaste15/miniconda3/envs/prostate39/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/uqaste15/miniconda3/envs/prostate39/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [19]:
# --- Data preparation for bids-2024 and bids-2025 ---
# The script collects subjects from both datasets and combines the modality lists.
bids_dirs = ["bids-2025-2"]

# Setup dictionary to hold paths for each subject
# This will be used to create a pandas dataframe later
# Initialize an empty list to hold the paths
paths_list = []

for bids_dir in bids_dirs:
    subject_dirs = sorted(glob.glob(os.path.join(bids_dir, "z*")))

    for subject_dir in subject_dirs:
        subject_id = os.path.basename(subject_dir)

        # Check if roi_niftis_mri_space/ directory exists
        roi_dir = os.path.join(subject_dir, "roi_niftis_mri_space")
        if not os.path.exists(roi_dir):
            print(f"Skipping {subject_id} as roi_niftis_mri_space directory does not exist.")
            continue

        # Get all nifti files
        nii_paths = glob.glob(os.path.join(subject_dir, "*.nii*"))

        # Get all nifti files in the roi_niftis_mri_space directory
        nii_paths += glob.glob(os.path.join(roi_dir, "*.nii*"))

        paths_dict = {}

        for nii_path in nii_paths:
            paths_dict['subject_id'] = subject_id
            file_name = os.path.basename(nii_path).split('.')[0]

            if 'GS1' in file_name:
                segmentation_name = "GS1"
            elif 'GS2' in file_name:
                segmentation_name = "GS2"
            elif 'GS3' in file_name:
                segmentation_name = "GS3"
            else:
                segmentation_name = "_".join(file_name.split('_')[1:])

            if segmentation_name == "roi_CTV_High_MR":
                segmentation_name = "Prostate"
            
            paths_dict[segmentation_name] = nii_path

        # Append the dictionary to the list
        paths_list.append(paths_dict)

print(paths_list)
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(paths_list)
# Replace NaN values with None
df = df.where(pd.notnull(df), None)

# For rows with no Prostate segmentation, if there is a roi_CTV_Low_MR segmentation, use it as Prostate
for index, row in df.iterrows():
    if row.get('Prostate') is None and row.get('roi_CTV_Low_MR') is not None:
        df.at[index, 'Prostate'] = row['roi_CTV_Low_MR']

for index, row in df.iterrows():
    if row.get('Prostate') is None and row.get('roi_CTVp_MR') is not None:
        df.at[index, 'Prostate'] = row['roi_CTVp_MR']

# Remove all columns except
columns_to_keep = ['subject_id', 'MRI', 'GS1', 'GS2', 'GS3', 'MRI_homogeneity-corrected', 'CT', 'seeds', 'Prostate']
df = df[columns_to_keep]

df

Skipping z0117672 as roi_niftis_mri_space directory does not exist.
Skipping z0354148-Missing-T1 as roi_niftis_mri_space directory does not exist.
Skipping z0791295-Missing-T1 as roi_niftis_mri_space directory does not exist.
Skipping z3121144 as roi_niftis_mri_space directory does not exist.
[{'subject_id': 'z0110968', 'MRI': 'bids-2025-2/z0110968/z0110968_MRI.nii', 'MRI_homogeneity-corrected': 'bids-2025-2/z0110968/z0110968_MRI_homogeneity-corrected.nii', 'CT': 'bids-2025-2/z0110968/z0110968_CT.nii', 'seeds': 'bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_seeds.nii.gz', 'GS1': 'bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_roi_GS1_MR.nii.gz', 'GS3': 'bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_roi_GS3_MR.nii.gz', 'GS2': 'bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_roi_GS2_MR.nii.gz', 'Prostate': 'bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_roi_CTV_High_MR.nii.gz', 'roi_zCTV_High_MR': 'bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_roi_zCTV_High_MR.n

subject_id                                    MRI  \
0    z0110968  bids-2025-2/z0110968/z0110968_MRI.nii   
1    z0155654  bids-2025-2/z0155654/z0155654_MRI.nii   
2    z0173340  bids-2025-2/z0173340/z0173340_MRI.nii   
3    z0177975  bids-2025-2/z0177975/z0177975_MRI.nii   
4    z0194267  bids-2025-2/z0194267/z0194267_MRI.nii   
5    z0236177  bids-2025-2/z0236177/z0236177_MRI.nii   
6    z0301597  bids-2025-2/z0301597/z0301597_MRI.nii   
7    z0313444  bids-2025-2/z0313444/z0313444_MRI.nii   
8    z0324824  bids-2025-2/z0324824/z0324824_MRI.nii   
9    z0352318  bids-2025-2/z0352318/z0352318_MRI.nii   
10   z0373678  bids-2025-2/z0373678/z0373678_MRI.nii   
11   z0412835  bids-2025-2/z0412835/z0412835_MRI.nii   
12   z0452629  bids-2025-2/z0452629/z0452629_MRI.nii   
13   z0457759  bids-2025-2/z0457759/z0457759_MRI.nii   
14   z0472759  bids-2025-2/z0472759/z0472759_MRI.nii   
15   z0713437  bids-2025-2/z0713437/z0713437_MRI.nii   
16   z0772718  bids-2025-2/z0772718/z0772718_MRI.nii   
17   z1149535  bids-2025-2/z1149535/z1149535_MRI.nii   
18   z1223140  bids-2025-2/z1223140/z1223140_MRI.nii   
19   z1260717  bids-2025-2/z1260717/z1260717_MRI.nii   
20   z1270838  bids-2025-2/z1270838/z1270838_MRI.nii   
21   z1290672  bids-2025-2/z1290672/z1290672_MRI.nii   
22   z1309292  bids-2025-2/z1309292/z1309292_MRI.nii   
23   z1314187  bids-2025-2/z1314187/z1314187_MRI.nii   
24   z1363713  bids-2025-2/z1363713/z1363713_MRI.nii   
25   z1390547  bids-2025-2/z1390547/z1390547_MRI.nii   
26   z1438488  bids-2025-2/z1438488/z1438488_MRI.nii   
27   z1451978  bids-2025-2/z1451978/z1451978_MRI.nii   
28   z1497141  bids-2025-2/z1497141/z1497141_MRI.nii   
29   z1504177  bids-2025-2/z1504177/z1504177_MRI.nii   
30   z1510293  bids-2025-2/z1510293/z1510293_MRI.nii   
31   z1521481  bids-2025-2/z1521481/z1521481_MRI.nii   
32   z1548312  bids-2025-2/z1548312/z1548312_MRI.nii   
33   z1549811  bids-2025-2/z1549811/z1549811_MRI.nii   
34   z1565042  bids-2025-2/z1565042/z1565042_MRI.nii   
35   z1588302  bids-2025-2/z1588302/z1588302_MRI.nii   
36   z1589675  bids-2025-2/z1589675/z1589675_MRI.nii   
37   z1625220  bids-2025-2/z1625220/z1625220_MRI.nii   
38   z1627398  bids-2025-2/z1627398/z1627398_MRI.nii   
39   z1719739  bids-2025-2/z1719739/z1719739_MRI.nii   
40   z1799707  bids-2025-2/z1799707/z1799707_MRI.nii   
41   z1818331  bids-2025-2/z1818331/z1818331_MRI.nii   
42   z1859491  bids-2025-2/z1859491/z1859491_MRI.nii   
43   z1962526  bids-2025-2/z1962526/z1962526_MRI.nii   
44   z2034911  bids-2025-2/z2034911/z2034911_MRI.nii   
45   z2716526  bids-2025-2/z2716526/z2716526_MRI.nii   
46   z2792304  bids-2025-2/z2792304/z2792304_MRI.nii   
47   z2799618  bids-2025-2/z2799618/z2799618_MRI.nii   
48   z3013142  bids-2025-2/z3013142/z3013142_MRI.nii   
49   z3137578  bids-2025-2/z3137578/z3137578_MRI.nii   
50   z3146117  bids-2025-2/z3146117/z3146117_MRI.nii   
51   z3154324  bids-2025-2/z3154324/z3154324_MRI.nii   
52   z3179174  bids-2025-2/z3179174/z3179174_MRI.nii   
53   z3264469  bids-2025-2/z3264469/z3264469_MRI.nii   
54   z3267901  bids-2025-2/z3267901/z3267901_MRI.nii   
55   z3278008  bids-2025-2/z3278008/z3278008_MRI.nii   
56   z3282235  bids-2025-2/z3282235/z3282235_MRI.nii   
57   z3284359  bids-2025-2/z3284359/z3284359_MRI.nii   
58   z6025869  bids-2025-2/z6025869/z6025869_MRI.nii   
59   z6029265  bids-2025-2/z6029265/z6029265_MRI.nii   

                                                  GS1  \
0   bids-2025-2/z0110968/roi_niftis_mri_space/z011...   
1   bids-2025-2/z0155654/roi_niftis_mri_space/z015...   
2   bids-2025-2/z0173340/roi_niftis_mri_space/z017...   
3   bids-2025-2/z0177975/roi_niftis_mri_space/z017...   
4   bids-2025-2/z0194267/roi_niftis_mri_space/z019...   
5   bids-2025-2/z0236177/roi_niftis_mri_space/z023...   
6   bids-2025-2/z0301597/roi_niftis_mri_space/z030...   
7   bids-2025-2/z0313444/roi_niftis_mri_space/z031...   
8   bids-2025-2/z0324824/roi_niftis_mri_space/z032...   
9   bids-2025-2/

In [23]:
# Here we combine GS1, GS2, and GS3 into a single NIfTI file as roi_niftis_mri_space/z..._seeds

# For each subject
for index, row in df.iterrows():
    subject_id = row['subject_id']
    print(f"Processing subject {subject_id}")

    #if os.path.exists(output_path):
    #    print(f"Output file {output_path} already exists. Skipping.")
    #    continue
    
    # Filter out None values
    seg_paths = [row['GS1'], row['GS2'], row['GS3']]
    seg_paths = [path for path in seg_paths if path is not None]

    # define output path alongside the first GS file
    output_path = os.path.join(os.path.dirname(seg_paths[0]), f"{subject_id}_seeds.nii.gz")
    
    
    if len(seg_paths) == 0:
        print(f"No seg files found for {subject_id}. Skipping.")
        continue
    
    # Load the first GS file to get the shape and affine
    gs_img = nib.load(seg_paths[0])
    combined_data = np.zeros(gs_img.shape, dtype=np.uint8)

    # 1) mark prostate voxels as class 2
    if row['Prostate'] is not None:
        prostate_data = nib.load(row['Prostate']).get_fdata() > 0
        combined_data[prostate_data] = 2

    # 2) mark *any* GS1/GS2/GS3 voxel as class 1
    for gs_path in seg_paths:
        gs_data = nib.load(gs_path).get_fdata() > 0
        combined_data[gs_data] = 1
    
    # Create a new NIfTI image
    combined_img = nib.Nifti1Image(combined_data, gs_img.affine)

    print(np.unique(combined_data, return_counts=True))
    
    # Save the combined image
    print(f"Saving combined seeds for {subject_id} to {output_path}")
    nib.save(combined_img, output_path)

Processing subject z0110968
(array([0, 1, 2], dtype=uint8), array([4051858,     591,   43551]))
Saving combined seeds for z0110968 to bids-2025-2/z0110968/roi_niftis_mri_space/z0110968_seeds.nii.gz
Processing subject z0155654
(array([0, 1, 2], dtype=uint8), array([4039266,     524,   56210]))
Saving combined seeds for z0155654 to bids-2025-2/z0155654/roi_niftis_mri_space/z0155654_seeds.nii.gz
Processing subject z0173340
(array([0, 1, 2], dtype=uint8), array([4070293,     976,   24731]))
Saving combined seeds for z0173340 to bids-2025-2/z0173340/roi_niftis_mri_space/z0173340_seeds.nii.gz
Processing subject z0177975
(array([0, 1, 2], dtype=uint8), array([4011101,    1196,   83703]))
Saving combined seeds for z0177975 to bids-2025-2/z0177975/roi_niftis_mri_space/z0177975_seeds.nii.gz
Processing subject z0194267
(array([0, 1, 2], dtype=uint8), array([4036405,     389,   59206]))
Saving combined seeds for z0194267 to bids-2025-2/z0194267/roi_niftis_mri_space/z0194267_seeds.nii.gz
Processing

In [6]:

def apply_homogeneity_correction(image_data):
    # Convert the NumPy array to a SimpleITK image.
    sitk_image = sitk.GetImageFromArray(image_data)
    # Create a mask using Otsu thresholding.
    mask = sitk.OtsuThreshold(sitk_image, 0, 1, 200)
    # Run the N4 bias field correction.
    corrected_image = sitk.N4BiasFieldCorrection(sitk_image, mask)
    # Convert the corrected SimpleITK image back to a NumPy array.
    corrected_data = sitk.GetArrayFromImage(corrected_image)
    return corrected_data

def append_suffix_to_filename(filename, suffix):
    # Handle files ending with '.nii.gz'
    if filename.endswith('.nii.gz'):
        base = filename[:-7]
        return base + suffix + '.nii.gz'
    else:
        base, ext = os.path.splitext(filename)
        return base + suffix + ext

# Process each MRI image.
for t1_file in df['MRI'].dropna():
    new_file = append_suffix_to_filename(t1_file, '_homogeneity-corrected')
    
    if os.path.exists(new_file):
        print(f"Skipping {t1_file} because {new_file} already exists.")
        continue

    nii = nib.load(t1_file)
    data = nii.get_fdata()
    corrected_data = apply_homogeneity_correction(data)
    
    corrected_img = nib.Nifti1Image(corrected_data, nii.affine, nii.header)
    nib.save(corrected_img, new_file)
    print(f"Saved corrected image to {new_file}")

Skipping bids-2025-2/z0110968/z0110968_MRI.nii because bids-2025-2/z0110968/z0110968_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0155654/z0155654_MRI.nii because bids-2025-2/z0155654/z0155654_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0173340/z0173340_MRI.nii because bids-2025-2/z0173340/z0173340_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0177975/z0177975_MRI.nii because bids-2025-2/z0177975/z0177975_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0194267/z0194267_MRI.nii because bids-2025-2/z0194267/z0194267_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0236177/z0236177_MRI.nii because bids-2025-2/z0236177/z0236177_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0301597/z0301597_MRI.nii because bids-2025-2/z0301597/z0301597_MRI_homogeneity-corrected.nii already exists.
Skipping bids-2025-2/z0313444/z0313444_MRI.nii because bids-2025-2/z0313444/z031344

In [116]:
from nilearn.image import resample_to_img

def resample_image_to_target(source_nii, target_nii, interpolation='continuous'):
        resampled_source = resample_to_img(
            source_nii,
            target_nii,
            interpolation=interpolation,
            force_resample=True,
            copy_header=True
        )

        # Acquire the spacing (zoom values) from the 
        target_spacing = target_nii.header.get_zooms()[:3]

        # Get image data and force the affine and zooms to match the target
        resampled_data = resampled_source.get_fdata()
        new_img = nib.Nifti1Image(resampled_data, target_nii.affine, target_nii.header)
        new_img.header.set_zooms(target_spacing)
        
        return new_img

# turn the above into a function
def resample_all_images(df, source_col, target_col, new_filename, interpolation='nearest'):
    for index, row in df.iterrows():
        source_nii = row[source_col]
        target_nii = row[target_col]

        if source_nii and target_nii:
            # Load the images
            source_img = nib.load(source_nii)
            target_img = nib.load(target_nii)

            # Resample the source image to the target image
            source_resampled = resample_image_to_target(source_img, target_img, interpolation)

            # Copy the header and affine from the target image
            source_resampled = nib.Nifti1Image(
                source_resampled.get_fdata(),
                target_img.affine,
                target_img.header
            )

            # Save the resampled image
            out_filepath = os.path.join(os.path.dirname(source_nii), new_filename)
            nib.save(source_resampled, out_filepath)
            print(f"Saved resampled image to {out_filepath}")

In [117]:
# Resample CT segmentations to QSM
resample_all_images(
    df=df,
    source_col='t1_segmentation',
    target_col='qsm_siemens',
    new_filename='qsm_siemens_segmentation.nii',
    interpolation='nearest'
)

Saved resampled image to bids-2024/sub-z0002292/ses-20240318/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0163277/ses-20240123/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0206923/ses-20231003/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0232771/ses-20240228/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0347933/ses-20240129/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0381949/ses-20231005/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0742379/ses-20240124/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0747844/ses-20240304/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z1318033/ses-20230825/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z1384048/ses-20231108/extra_data/qsm_siemens_segment

# Clean up segmentations

In [118]:
import enum
import numpy as np
import nibabel as nib
import os
from scipy.ndimage import (
    label,
    binary_dilation,
    generate_binary_structure,
    center_of_mass
)
import pandas as pd

# Define segmentation types
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

def create_spherical_mask(center, shape, radius):
    indices = np.indices(shape).transpose((1, 2, 3, 0))
    distances = np.linalg.norm(indices - center, axis=-1)
    return distances <= radius

def enforce_2d_single_component(mask):
    structure2d = generate_binary_structure(2, 1)  # 4-connected in 2D
    new_mask = mask.copy()
    for axis in range(3):
        for idx in range(mask.shape[axis]):
            if axis == 0:
                slice_mask = new_mask[idx, :, :]
            elif axis == 1:
                slice_mask = new_mask[:, idx, :]
            elif axis == 2:
                slice_mask = new_mask[:, :, idx]
            labeled_slice, num_features = label(slice_mask, structure=structure2d)
            if num_features > 1:
                sizes = [np.sum(labeled_slice == comp_id) for comp_id in range(1, num_features + 1)]
                largest_comp = np.argmax(sizes) + 1
                new_slice = (labeled_slice == largest_comp)
                if axis == 0:
                    new_mask[idx, :, :] = new_slice
                elif axis == 1:
                    new_mask[:, idx, :] = new_slice
                elif axis == 2:
                    new_mask[:, :, idx] = new_slice
    return new_mask

def gaussian_filter3d(input_data, sigma):
    from scipy.ndimage import gaussian_filter
    return gaussian_filter(input_data, sigma=sigma)

def clean_segmentation(seg_nii, input_nii, stdevs=1, calc_stdevs=1.2):
    # Load the segmentation and input image data
    seg = np.array(np.round(seg_nii.get_fdata()), dtype=np.uint8)
    input_data = input_nii.get_fdata()

    seed_mask_labelled, num_labels = label(
        input=(seg == SegType.GOLD_SEED.value),
        structure=np.ones((3, 3, 3))
    )
    seed_mask = np.zeros(seg.shape, dtype=bool)

    # Get prostate values
    prostate_values = input_data[seg == SegType.PROSTATE.value]

    for seed_id in range(1, num_labels + 1):
        seed_i_mask = (seed_mask_labelled == seed_id)
        coords = np.column_stack(np.where(seed_i_mask))
        min_voxel = tuple(coords[np.argmin(input_data[seed_i_mask])])
        centre_voxel = center_of_mass(seed_i_mask)
        middle_voxel = np.array([
            int((min_voxel[0] + centre_voxel[0]) / 2),
            int((min_voxel[1] + centre_voxel[1]) / 2),
            int((min_voxel[2] + centre_voxel[2]) / 2)
        ])
        seed_i_mask = create_spherical_mask(middle_voxel, input_data.shape, radius=4)
        
        seed_i_mask = seed_i_mask & (input_data < (prostate_values.mean() - stdevs * np.std(prostate_values)))

        # create a 3D T structure that avoids diagonal connections
        structure = np.array([
        [
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ],[ 
            [0, 1, 0],
            [1, 1, 1],
            [0, 1, 0]
        ],[
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ]], dtype=bool)

        seed_i_mask_labelled, num_labels_comp = label(seed_i_mask, structure=structure)
        if num_labels_comp > 1:
            sizes = [np.sum(seed_i_mask_labelled == label_id) for label_id in range(1, num_labels_comp + 1)]
            largest_label = np.argmax(sizes) + 1
            seed_i_mask = (seed_i_mask_labelled == largest_label)

        max_iters = 2
        iter_idx = 0
        current_stdevs = stdevs
        accum_mask = seed_i_mask.copy()

        while iter_idx < max_iters:
            candidate = binary_dilation(accum_mask)
            candidate_new = candidate & (input_data < (np.mean(prostate_values) - current_stdevs * np.std(prostate_values))) & (~accum_mask)
            if not candidate_new.any():
                break
            new_mask = accum_mask | candidate_new
            new_mask = enforce_2d_single_component(new_mask)
            if np.array_equal(new_mask, accum_mask):
                break
            accum_mask = new_mask
            iter_idx += 1
            current_stdevs += 1
        seed_i_mask = accum_mask
        seed_mask = np.logical_or(seed_mask, seed_i_mask)

    seed_mask_indices = np.argwhere(seed_mask)
    if seed_mask_indices.size == 0:
        seed_mask_center = np.array([input_data.shape[0] // 2, input_data.shape[1] // 2, input_data.shape[2] // 2])
        max_distance = 0
    else:
        min_indices = np.min(seed_mask_indices, axis=0)
        max_indices = np.max(seed_mask_indices, axis=0)
        seed_mask_center = np.array([
            int((min_indices[0] + max_indices[0]) / 2),
            int((min_indices[1] + max_indices[1]) / 2),
            int((min_indices[2] + max_indices[2]) / 2)
        ])
        max_distance = 0
        seed_mask_labelled, num_seed_labels = label(seed_mask, structure=np.ones((3, 3, 3)))
        for seed_id in range(1, num_seed_labels + 1):
            cur_seed_mask = (seed_mask_labelled == seed_id)
            cur_center = center_of_mass(cur_seed_mask)
            distance = np.linalg.norm(seed_mask_center - cur_center)
            if distance > max_distance:
                max_distance = distance

    calc_mask = input_data < (prostate_values.mean() - calc_stdevs * np.std(prostate_values))
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)

    calc_mask = binary_dilation(calc_mask, iterations=1)
    calc_mask = gaussian_filter3d(calc_mask.astype(np.float32), sigma=calc_stdevs)
    calc_mask = calc_mask > 0.5

    calc_mask_labelled, num_calc = label(calc_mask, structure=np.ones((3, 3, 3)))
    for calc_id in range(1, num_calc + 1):
        if np.sum(calc_mask_labelled == calc_id) == 1:
            calc_mask[calc_mask_labelled == calc_id] = False

    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0
    seg[calc_mask] = 2
    seg[seed_mask] = 1

    seg_clean_nii = nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine)
    return seg_clean_nii

# turn the above into a function
def clean_all_segmentations(df, input_col, seg_col, new_filename):
    for index, row in df.iterrows():
        input_path = row[input_col]
        seg_path = row[seg_col]

        if input_path and seg_path:
            # Load the images
            input_img = nib.load(input_path)
            seg_img = nib.load(seg_path)

            # Clean the segmentation
            cleaned_segmentation = clean_segmentation(seg_img, input_img)

            # Save the cleaned segmentation
            out_filepath = os.path.join(os.path.dirname(input_path), new_filename)
            nib.save(cleaned_segmentation, out_filepath)
            print(f"Saved cleaned segmentation to {out_filepath}")

In [119]:
# Use the above function instead
clean_all_segmentations(
    df=df,
    input_col='qsm_siemens',
    seg_col='qsm_siemens_segmentation',
    new_filename='qsm_siemens_segmentation_clean.nii'
)

Saved cleaned segmentation to bids-2024/sub-z0002292/ses-20240318/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0163277/ses-20240123/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0206923/ses-20231003/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0232771/ses-20240228/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0347933/ses-20240129/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0381949/ses-20231005/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0742379/ses-20240124/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0747844/ses-20240304/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z1318033/ses-20230825/extra_data/qsm_siemens_segmentation_cl

# Combine magnitude images

In [ ]:
for session_dir in session_dirs:
    mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*run-01*part-mag*nii*")))
    nii = nib.load(mag_images[0])
    mag_4d = np.array([nib.load(mag_images[i]).get_fdata() for i in range(len(mag_images))])
    mag_combined = np.sqrt(np.sum(np.square(mag_4d), axis=0))
    filename = os.path.join(session_dir, "extra_data", "magnitude_combined.nii")
    nib.save(nib.Nifti1Image(mag_combined, header=nii.header, affine=nii.affine), filename)
    print(f"Saved {filename}")

# Generate DICOMs for Radiographers

In [ ]:
for session_dir in session_dirs[:4]:
    subject_id = session_dir.split(os.sep)[1]
    
    B0_file = [B0_file for B0_file in fmap_files if subject_id in B0_file][0]
    qsm_file = [qsm_file for qsm_file in qsm_files if subject_id in qsm_file][0]
    mag_file = [mag_file for mag_file in mag_files if subject_id in mag_file][0]
    
    !mkdir -p {os.path.join(session_dir, "extra_data", "B0_dcm")}
    !mkdir -p {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")}
    !mkdir -p {os.path.join(session_dir, "extra_data", "qsm_dcm")}
    
    !nii2dcm --dicom_type MR --centered {B0_file} {os.path.join(session_dir, "extra_data", "B0_dcm")}
    !nii2dcm --dicom_type MR {mag_file} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")}
    !nii2dcm --dicom_type MR --centered {qsm_file} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

    !tar cf {subject_id}.tar {os.path.join(session_dir, "extra_data", "B0_dcm")} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

    !rm -rf {os.path.join(session_dir, "extra_data", "B0_dcm")} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")} {os.path.join(session_dir, "extra_data", "qsm_dcm")}